In [2]:
import numpy as np
import pandas as pd

In [3]:
#https://www.kaggle.com/irfanasrullah/groceries
df = pd.read_csv("compras.csv", header=None)
df.replace(np.nan, 0, inplace = True)

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,citrus fruit,semi-finished bread,margarine,ready soups,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tropical fruit,yogurt,coffee,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,whole milk,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,pip fruit,yogurt,cream cheese,meat spreads,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,other vegetables,whole milk,condensed milk,long life bakery product,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def removeAllOcurrencesOfValueInList(_list, value):
    return list(filter(lambda x: x != value, _list))

In [10]:
list_all_transactions = []

for index, row in df.iterrows():
    list_single_transaction = row.values.tolist()
    list_single_transaction = removeAllOcurrencesOfValueInList(list_single_transaction, 0)
    list_all_transactions.append(list_single_transaction)

#10 primeiras transações
print(list_all_transactions[0:10])

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'], ['tropical fruit', 'yogurt', 'coffee'], ['whole milk'], ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads'], ['other vegetables', 'whole milk', 'condensed milk', 'long life bakery product'], ['whole milk', 'butter', 'yogurt', 'rice', 'abrasive cleaner'], ['rolls/buns'], ['other vegetables', 'UHT-milk', 'rolls/buns', 'bottled beer', 'liquor (appetizer)'], ['potted plants'], ['whole milk', 'cereals']]


In [12]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
#http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/
te_ary = te.fit(list_all_transactions).transform(list_all_transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

df.head()

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [16]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(df, min_support = 0.01, use_colnames = True)
frequent_itemsets.sort_values(by=['support'], ascending = False).head(10)

,support,itemsets
86,0.255516,(whole milk)
55,0.193493,(other vegetables)
66,0.183935,(rolls/buns)
75,0.174377,(soda)
87,0.139502,(yogurt)
6,0.110524,(bottled water)
67,0.108998,(root vegetables)
81,0.104931,(tropical fruit)
73,0.098526,(shopping bags)
70,0.093950,(sausage)


In [17]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
rules.sort_values(by=['lift'], ascending = False).drop(['antecedent support', 'consequent support', 'leverage', 'conviction'], axis=1)

,antecedents,consequents,support,confidence,lift
74,"(other vegetables, citrus fruit)",(root vegetables),0.010371,0.359155,3.295045
96,"(other vegetables, tropical fruit)",(root vegetables),0.012303,0.342776,3.144780
1,(beef),(root vegetables),0.017387,0.331395,3.040367
73,"(root vegetables, citrus fruit)",(other vegetables),0.010371,0.586207,3.029608
95,"(root vegetables, tropical fruit)",(other vegetables),0.012303,0.584541,3.020999
...,...,...,...,...,...
65,(waffles),(whole milk),0.012710,0.330688,1.294196
17,(coffee),(whole milk),0.018709,0.322242,1.261141
61,(sausage),(whole milk),0.029893,0.318182,1.245252
6,(bottled water),(whole milk),0.034367,0.310948,1.216940


In [ ]:
#https://www.geeksforgeeks.org/implementing-apriori-algorithm-in-python/
#https://datascience.stackexchange.com/questions/16769/best-frequent-itemset-package-in-python
#http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/